## p-y for sand

# Notes

Need to run the code below again to reinstall the package

```shell
pip install -e . 
```

The code has been moved to the source folder for house keeping

In [1]:
from src.cpt import CPT
from pathlib import Path
import os
import plotly.io as pio
import numpy as np
import pandas as pd
from numpy import degrees, log10, pi, radians, tan, sin, cos
from src.p_y_sand import *
from src.p_y_clay import *
from src.pile import PipePile
import unittest
from src.p_y_unittests import *
import plotly.graph_objs as go

### Import data from CPT file
#### - Data format - 

`key` is the identifier in the txt file, for locating the data, it shoudl be a unit identifier just above the data
`patter` - an regular experss to specify the pattern of the data, you can see the example below

After reading the data, need to change the column name, so that all calculation can know the location of the data

#### - Rename Columns - 
```python
cpt_data.update_data_column_name(['Reading','SCPT_DPTH','SCPT_RES','SCPT_FRES','SCPT_PWP2','slopex','slopey'])
```

- `SCPT_DPTH` - depth below seabed
- `SCPT_RES` - raw `qc` value
- `SCPT_FRES` - shaft resistance $f_s$
- `SCPT_RES` - tip resistance $q_c$

#### - Specify Unit - 

Need to update the unit, differnet contactor may output data in different unit, this is one by 

```python
cpt_data.set_data_unit(['MPa','MPa','MPa'])
```

unit is assigned in a sequence [$q_c$, $f_s$,$u_2$]

#### - init_CPT - 

This is calculate $\gamma_{soil}$ and calculate the overburnden stress $\sigma_v$ 


In [2]:
filename ='CPT4.A00'
fileloc = Path(r'G:\Name Folders\Current Staff\CC213\10 IiA funding\Unified CPT Methods\01 Data\CPT Data\18 0091 05 R002 CPT ASCII')

cpt_data = CPT()
cpt_data.read_ASCII(fileloc / filename,key='Data--',pattern= r'(.{7})(.{10})(.{11})(.{11})(.{11})(.{11})(.{11})')
cpt_data.update_data_column_name(['Reading','SCPT_DPTH','SCPT_RES','SCPT_FRES','SCPT_PWP2','slopex','slopey'])
cpt_data.set_data_unit(['MPa','MPa','MPa'])
cpt_data.init_CPT()

23-10-25 19:07:09 -c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:327 DEBUG - Data unit is set to ['MPa', 'MPa', 'MPa']
c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:591: RuntimeWarning:

invalid value encountered in log10

23-10-25 19:07:11 -c:\Users\Emma.Shi\Desktop\CPT\UnitCPT\src\cpt.py:156 DEBUG - qt calculated using net area ratio of 0.85


### Input external parameters

In [3]:
Diameter = 2.0
Loading = 'Cyclic' # 'Monotonic' or 'Cyclic'
nkt = 12
N1 = 12
N2 = 3.22
interval = 1 #depth interval for sampling the raw CPT data
y_interval = 15 #identify the y[i] of p-y curve for sand
Isotropy = 'true'
I_p = 35
Clay_type = 'Gulf of Mexico' #identify clay type for cyclic p-y curves: 'Gulf of Mexico', 'North Sea soft clay', 'North Sea stiff clay'

### Resample cpt_data

In [4]:
resampled_cpt_data = interpolate_cpt_data(cpt_data.df, interval)

In [5]:
resampled_cpt_data['SCPT_DPTH']

0      0
1      1
2      2
3      3
4      4
      ..
70    70
71    71
72    72
73    73
74    74
Name: SCPT_DPTH, Length: 75, dtype: int32

### Determine soil type based on Ic
Ic<2.6 for sand, >2.6 for clay

In [6]:
resampled_cpt_data ['soil_type'] = resampled_cpt_data.apply(lambda row:determine_soil_type(ic = row['Ic']), axis = 1)

### Calculate p-y parameters for sand

In [7]:
resampled_cpt_data ['phi_e'] = resampled_cpt_data.apply(lambda row:calc_phi_e(qt = row['qt'], sigma_v = row['sigma_v'], sigma_v_e = row['sigma_v_e']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['C1'] = resampled_cpt_data.apply(lambda row:calc_C1(phi_e = row['phi_e']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['C2'] = resampled_cpt_data.apply(lambda row:calc_C2(phi_e = row['phi_e']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['C3'] = resampled_cpt_data.apply(lambda row:calc_C3(phi_e = row['phi_e']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['k'] = resampled_cpt_data.apply(lambda row:calc_k(phi_e = row['phi_e']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['pr'] = resampled_cpt_data.apply(lambda row:calc_pr(D = Diameter, gamma = row['gamma'], z = row['SCPT_DPTH'], C1 = row['C1'], C2 = row['C2'], C3 = row['C3']) if row['soil_type'] == 'sand' else None, axis = 1)
resampled_cpt_data ['A'] = resampled_cpt_data.apply(lambda row:calc_A(D = Diameter, z = row['SCPT_DPTH'], loading = Loading) if row['soil_type'] == 'sand' else None, axis = 1)

### Calculate undrained shear strength (su) for clay

In [8]:
resampled_cpt_data ['su'] = resampled_cpt_data.apply(lambda row:calc_su(qt = row['qt'], sigma_v = row['sigma_v'], nkt = nkt) if row['soil_type'] == 'clay' else None, axis = 1)

### Identify initial undrained shear strength (su0) for each clay layer

In [9]:
identify_soil_layers(resampled_cpt_data)

### Calculate p-y parameters for clay

In [10]:
resampled_cpt_data ['su1'] = resampled_cpt_data.apply(lambda row:calc_su1(su = row['su'], su0 = row['su0'], z= row['SCPT_DPTH']) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['alpha'] = resampled_cpt_data.apply(lambda row:calc_alpha(su = row['su'], sigma_v_e = row['sigma_v_e']) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['N_pd'] = resampled_cpt_data.apply(lambda row:calc_N_pd(alpha = row['alpha']) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['d'] = resampled_cpt_data.apply(lambda row:calc_d(su0 = row['su0'], su1 = row['su1'], D = Diameter) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['N_p0'] = resampled_cpt_data.apply(lambda row:calc_N_p0(N_1 = N1, N_2 = N2, alpha = row['alpha'], d = row['d'], D = Diameter, N_pd = row['N_pd'], z = row['SCPT_DPTH']) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['N_P'] = resampled_cpt_data.apply(lambda row:calc_N_P(N_pd = row['N_pd'], N_p0 = row['N_p0'], gamma = row['gamma'], z = row['SCPT_DPTH'], su = row['su'], isotropy = Isotropy) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['pu'] = resampled_cpt_data.apply(lambda row:calc_pu(su = row['su'], D = Diameter, N_P = row['N_P']) if row['soil_type'] == 'clay' else None, axis = 1)
resampled_cpt_data ['OCR'] = resampled_cpt_data.apply(lambda row:calc_OCR(Qt1 = row['Qt1']) if row['soil_type'] == 'clay' else None, axis = 1)

### Generate p-y curves

In [11]:
for i in range(12):
    resampled_cpt_data [f'y{i}'] = resampled_cpt_data.apply(lambda row:calc_y_mo(I_p = I_p, OCR = row['OCR'], D= Diameter, id_p = i) if row['soil_type'] == 'clay' else calc_y(y_interval = y_interval, i = i), axis = 1)
    resampled_cpt_data [f'p{i}'] = resampled_cpt_data.apply(lambda row:calc_p_mo(pu = row['pu'], id_p = i) if row['soil_type'] == 'clay' else calc_p(y = row[f'y{i}'], A = row['A'], pr = row['pr'], z = row['SCPT_DPTH'], k = row['k']), axis = 1)

### Identify cyclic p-y curves for clay

In [12]:
if Loading == 'Cyclic':
   for i in range(12):
     resampled_cpt_data [f'h_f{i}'] = resampled_cpt_data.apply(lambda row:calc_h_f(p_mo = row[f'p{i}'], p_u = row['pu'], z= row['SCPT_DPTH'], D = Diameter) if row['soil_type'] == 'clay' else None, axis = 1)
     resampled_cpt_data [f'N_eq{i}'] = resampled_cpt_data.apply(lambda row:calc_N_eq(h_f = row[f'h_f{i}'], clay_type = Clay_type) if row['soil_type'] == 'clay' else None, axis = 1)
     resampled_cpt_data [f'p_cy{i}'] = resampled_cpt_data.apply(lambda row:calc_p_y_mod(N_eq = row[f'N_eq{i}'], clay_type = Clay_type)[0] * row[f'p{i}'] if row['soil_type'] == 'clay' else None, axis = 1)
     resampled_cpt_data [f'y_cy{i}'] = resampled_cpt_data.apply(lambda row:calc_p_y_mod(N_eq = row[f'N_eq{i}'], clay_type = Clay_type)[1] * row[f'y{i}'] if row['soil_type'] == 'clay' else None, axis = 1) 

### Export results and plot figures

In [13]:
export_p_y_sand(resampled_cpt_data, "cpt_data_resampled.xlsx")
plot_p_y_curve(resampled_cpt_data, 8)

cpt_data_resampled.xlsx has been exported successfully.


In [14]:
plot_p_y_cyclic(resampled_cpt_data, 8)

In [27]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output

# Define the x_mo and y_mo variables
x_mo = [1, 2, 3, 4, 5]
y_mo = [2, 4, 6, 8, 10]

# Create the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1('My Dashboard'),
    dcc.Graph(id='my-graph'),
    html.Button('Show/hide markers', id='marker-button', n_clicks=0)
])

# Define the callback function that updates the graph
@app.callback(
    Output('my-graph', 'figure'),
    [Input('marker-button', 'n_clicks')]
)
def update_graph(n_clicks):
    # Create a scatter plot trace with the x_mo and y_mo variables
    trace = dict(
        x=x_mo,
        y=y_mo,
        mode='markers',
        marker=dict(size=10, color='blue', opacity=0.5)
    )

    # Add or remove markers based on button click
    if n_clicks % 2 == 0:
        trace['marker']['opacity'] = 0
    else:
        trace['marker']['opacity'] = 0.5

    # Create the figure object and return it
    fig = dict(data=[trace])
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

c:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



### Unittests

In [15]:
# Create a TestSuite object
suite = unittest.TestSuite()

# Add the test_p_y_sand_monotonic method to the TestSuite
suite.addTest(TestUnifiedCPT('test_p_y_sand_monotonic'))
suite.addTest(TestUnifiedCPT('test_p_y_sand_cyclic'))
suite.addTest(TestUnifiedCPT('test_p_y_clay_monotonic'))

# Create a TestRunner object
runner = unittest.TextTestRunner()

# Run the TestSuite using the TestRunner
runner.run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [16]:
def identify_soil_layers(cpt_data):
    # Initialize variables
    prev_soil_type = None
    layer_counts = []
    layer_thicknesses = []
    layer_types = []
    layer_strengths_0 = []

    # Iterate over rows in the DataFrame
    for i, row in cpt_data.iterrows():
        # Check if this row has a different soil type than the previous row
        if row['soil type'] != prev_soil_type:
            # If this is not the first layer, record the count, thickness, soil type, and initial strength of the previous layer
            if prev_soil_type is not None:
                layer_counts.append(layer_count)
                layer_thickness = cpt_data.loc[i-1, 'depth'] - cpt_data.loc[i-layer_count, 'depth']
                layer_thicknesses.append(layer_thickness)
                layer_types.append(prev_soil_type)
                layer_strengths_0.append(cpt_data.loc[i-layer_count, 'strength'])
            # Reset the layer count and update the previous soil type
            layer_count = 1
            prev_soil_type = row['soil type']
        else:
            # Increment the layer count if the soil type is the same as the previous row
            layer_count += 1

    # Record the count, thickness, soil type, and initial strength of the last layer
    layer_counts.append(layer_count)
    layer_thickness = cpt_data['depth'].max() - cpt_data.loc[len(cpt_data)-layer_count, 'depth']
    layer_thicknesses.append(layer_thickness)
    layer_types.append(prev_soil_type)
    layer_strengths_0.append(cpt_data.loc[len(cpt_data)-layer_count, 'strength'])

    # Add the 'strength_0' column to the DataFrame
    cpt_data['strength_0'] = pd.Series([val for val, count in zip(layer_strengths_0, layer_counts) for i in range(count)], index=cpt_data.index)

    # Append the 'strength_0' value to each row of the DataFrame
    for i, row in cpt_data.iterrows():
        row['strength_0'] = layer_strengths_0[layer_types.index(row['soil type'])]

    # Print the results
    print(cpt_data)

# Initialize sample DataFrame
cpt_data = pd.DataFrame({
    'depth': [0.0, 1.5, 3.0, 4.5, 6.0, 7.5, 9.0, 10.0, 12.0, 15.0],
    'soil type': ['clay', 'clay', 'sand', 'sand', 'sand', 'clay', 'clay', 'sand', 'clay', 'sand'],
    'strength': [10, 12, 20, 22, 24, 15, 18, 19, 20, 20]
})

# Call the identify_soil_layers function
identify_soil_layers(cpt_data)


   depth soil type  strength  strength_0
0    0.0      clay        10          10
1    1.5      clay        12          10
2    3.0      sand        20          20
3    4.5      sand        22          20
4    6.0      sand        24          20
5    7.5      clay        15          15
6    9.0      clay        18          15
7   10.0      sand        19          19
8   12.0      clay        20          20
9   15.0      sand        20          20


# Sampling the P-y points

** To-Do** 

[ ]Need to read original paper to workout the points. 

Appendix `A.8.5.2.1.3` provides more detailed discussion on the rational of determining the p-y points. It will be more elegant to implement the p-y curves for clay in this way

The p-y points are based on scaling the DSS stress-strain curves as follows:

$$
\begin{equation}
\frac{\tau}{s_u} = \frac{\tanh\bigg(a \cdot (\frac{\gamma}{\gamma_f})^{0.5}\bigg)}{\tanh{a}}
\end{equation}
$$

From the test data, the parameters in this equation will be fitted to get $a$ and $\gamma_f$

And the final p-y curve will be in the format of

$$
\begin{equation}
\frac{p}{p_u} = \frac{\tanh{\bigg[A\cdot \bigg(\frac{y/D}{y/D_f}\bigg)^{0.5}\bigg]}}{\tanh{A}}
\end{equation}
$$

where:

$ A = 1.33+0.45\cdot a$, and 

$(y/D)_f = \gamma_f (2.5-1.2\ln(a))$

$$
\begin{equation}
\frac{y}{D} = \bigg[\tanh^{-1}\bigg({\frac{p}{p_u}\tanh(A)}\bigg)/A\bigg]^2 \cdot (\frac{y}{D})_f
\end{equation}
$$
In the code $\gamma_f$, i.e., the shear strength at the failure is assumed to be 0.15 for all OCR while $a$ varies with OCR as below:

|OCR|$\gamma_f$|$a$|
|--|--:|--:|
|$\le 2$|0.15|2.38|
|4|0.15|1.5|
|10|0.15|1.0|

Assuming the hyperbolic function for $a$ fitted from the three points proposed.

``` python

f_model_a = lambda ocr: 3.457/ocr + 0.647

```


In [17]:
from scipy.interpolate import interp1d
def calc_y_mo(Ip, OCR, p_pu):
    gamma_f = 0.15 
    f_model_a = lambda ocr: 3.457/ocr + 0.647
    a = f_model_a(OCR)
    A = 1.33+0.45*a
    yD_f = gamma_f*(2.5-1.2*np.log(a))
    y_D = (1/A*np.arctanh(p_pu*np.tanh(A)))**2 *yD_f
    f_multiplier = interp1d(x=[2,4,10],y=[0.33,0.5,0.66],fill_value = (0.33, 0.66),bounds_error=False)
    if Ip>30:
        y_D_multiplier = f_multiplier([OCR])[0]
    else:
        y_D_multiplier  = 1.0
    return y_D*y_D_multiplier

In [18]:
import src
import plotly.graph_objects as go
p=np.array([0.000, 0.050, 0.200, 0.300, 0.400, 0.500, 0.600, 0.700, 0.800, 0.900, 0.975, 1.000])
fig = src.geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=2,p_pu=p),name='OCR=2'))
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=4,p_pu=p),name='OCR=4'))
fig.add_trace(go.Scatter(y=p, x=calc_y_mo(Ip=29,OCR=10,p_pu=p),name='OCR=10'))

In [19]:
df = pd.DataFrame()
df['pmo_pu'] =p
df['OCR=2 & Ip>30'] = calc_y_mo(35,2,p)
df['OCR=4 & Ip>30'] = calc_y_mo(35,4,p)
df['OCR=8 & Ip>30'] = calc_y_mo(35,8,p)

In [20]:
df

,pmo_pu,OCR=2 & Ip>30,OCR=4 & Ip>30,OCR=8 & Ip>30
0,0.000,0.000000,0.000000,0.000000
1,0.050,0.000030,0.000087,0.000150
2,0.200,0.000500,0.001421,0.002451
3,0.300,0.001163,0.003303,0.005692
4,0.400,0.002175,0.006163,0.010602
5,0.500,0.003645,0.010297,0.017668
6,0.600,0.005779,0.016243,0.027757
7,0.700,0.008981,0.025046,0.042525
8,0.800,0.014214,0.039072,0.065589
9,0.900,0.024630,0.065419,0.107095


In [21]:
import plotly.express as px
import plotly.graph_objects as go

In [22]:
import numpy as np
from scipy.optimize import curve_fit
def tanh_function(x, a, b):
    return a * 1/x + b

# Three example points
y_data = np.array([2.38,1.5,1.0])
x_data = np.array([2,4,10])

# Perform the curve fit
popt, _ = curve_fit(tanh_function, x_data, y_data)
print(popt)
model_a = lambda ocr: 3.457/ocr + 0.647

[3.45714286 0.64714286]


In [23]:
x_fit = np.linspace(min(x_data), max(x_data), 100)
y_fit = tanh_function(x_fit, *popt)

In [24]:
# fig = px.line(y=[2.38,2.38,1.5,1.0], x=[1,2,4,10])
x_data = np.array([2,4,10])
y_data = np.array([2.38,1.5,1])
from src import geoplot
fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_data,y=model_a(x_data),mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_fit,y=y_fit,name='Fitted Hyperbolic function'))
fig.update_layout(width=800,height=400)
fig.update_xaxes(title='ORC(>2.0)')
fig.update_yaxes(title='Coefficient (a)')

In [25]:
popt

array([3.45714286, 0.64714286])

In [26]:
import numpy as np
from scipy.optimize import curve_fit
def tanh_function(x, a,b):
    return a/x+b
# Three example points
x_data = np.array([2,4,10])
y_data = np.array([0.33,0.5,0.66])

# Perform the curve fit
popt, _ = curve_fit(tanh_function, x_data, y_data)
print(popt)

x_fit = np.linspace(min(x_data), max(x_data), 100)
y_fit = tanh_function(x_fit, *popt)
#
model_a = lambda ocr: 3.457/ocr + 0.647
from src import geoplot
fig = geoplot.GEOPlot.get_figure()
fig.add_trace(go.Scatter(x=x_data,y=y_data,mode='markers',name='ISO Points'))
fig.add_trace(go.Scatter(x=x_fit,y=y_fit,name='Fitted Hyperbolic function')) 
fig.update_layout(width=800,height=400)
fig.update_xaxes(title='ORC(>2.0)')
fig.update_yaxes(title='Coefficient (a)')

[-0.81020408  0.72622449]
